In [51]:
import argparse
import sys

In [52]:
parser = argparse.ArgumentParser(description="Generate a fake news headline in the current folder")
parser.add_argument("-s", "--sentiment", default='positive', type=str, help="Sentiment of generated news - positive or negative")
args = parser.parse_args()
sentiment = args.sentiment

usage: __main__.py [-h] [-s SENTIMENT]
__main__.py: error: unrecognized arguments: -f /Users/shairozsohail/Library/Jupyter/runtime/kernel-10abf7de-50d5-48b5-87fa-c95920c3647f.json


SystemExit: 2

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [49]:
def get_headline(seed='The', sentiment='positive'):
    #print("Creating model")
    # load json and create model
    if sentiment=='positive':
        json_file = open('positive_model.json', 'r')
    else:
        json_file = open('negative_model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    #loaded_model = model_from_json(loaded_model_json)

    # load weights into new model
    if sentiment=='positive':
        loaded_model.load_weights("positive_model.h5")
    else:
        loaded_model.load_weights("negative_model.h5")
    loaded_model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    #print("Loaded " + str(sentiment) + " model from disk")
    
    
    #print("Generating headline")
    for diversity in [0.2]:
        generated = ''
        sentence = seed
        generated += sentence
        #print('----- Generating with seed: "' + sentence + '"')
        #print()
        #sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars))) # Padding
            for t, char in enumerate(sentence): 
                x_pred[0, t, char_indices[char]] = 1.

            preds = loaded_model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            #sys.stdout.write(next_char)
            #sys.stdout.flush()
        return(sentence)
        
        
def get_image_of(query, filepath='image_result.jpg'):
    subscriptionKey = '058604aaaf914e7f9e46671f0a8c85ae'
    import http.client, urllib.parse, json
    # Verify the endpoint URI.  At this writing, only one endpoint is used for Bing
    # search APIs.  In the future, regional endpoints may be available.  If you
    # encounter unexpected authorization errors, double-check this value against
    # the endpoint for your Bing search instance in your Azure dashboard.
    host = "api.cognitive.microsoft.com"
    path = "/bing/v7.0/images/search"
    term = query.replace(' ', '+')

    def BingImageSearch(search):
        "Performs a Bing image search and returns the results."

        headers = {'Ocp-Apim-Subscription-Key': subscriptionKey}
        conn = http.client.HTTPSConnection(host)
        query = urllib.parse.quote(search)
        conn.request("GET", path + "?q=" + query, headers=headers)
        response = conn.getresponse()
        headers = [k + ": " + v for (k, v) in response.getheaders()
                       if k.startswith("BingAPIs-") or k.startswith("X-MSEdge-")]
        return headers, response

    #print('Searching images for: ', term)
    headers, result = BingImageSearch(term)
    
    response_dict = json.load(result)
    image_url = response_dict['value'][1]['contentUrl']
    #print('Saving image...')
    urllib.request.urlretrieve(image_url, filepath)


def generate_headline_document(headline_text, headline_image, filename='new_headline.pdf'):
    from reportlab.pdfgen import canvas
    from reportlab.lib.units import cm
    c = canvas.Canvas(filename)
    c.setFont('Helvetica-Bold', 26)
    c.drawString(80,750,headline_text)
    c.drawInlineImage(headline_image, x=80, y=420, preserveAspectRatio=True, width=10*cm, anchor='w')
    c.save()



In [47]:
def generate_fake_news(sentiment='positive', output_file='fake_news.pdf'):
    print("Generating " + sentiment + " headline")
    headline = get_headline(sentiment=sentiment)
    print("Generating images")
    image = get_image_of(headline)
    print("Generating news")
    generate_headline_document(headline_text=headline, headline_image=image)
    print("Fake News generated at " + output_file)

Searching images for:  milk+carton
Saving image...


In [48]:
generate_fake_news(sentiment=sentiment)

Searching images for:  call+for+success+on+us+communities+to+be+over+australia
Saving image...
